# Lab | Web Scraping

Welcome to the IMDb Web Scraping Adventure Lab!

**Objective**

In this lab, we will embark on a mission to unearth valuable insights from the vast sea of data available on IMDb, one of the largest online databases of movie, TV, and celebrity information. As budding data scientists and business analysts, you have been tasked to scrape a specific subset of data from IMDb to assist film production companies in understanding the landscape of highly-rated movies in a defined time period. Your insights will potentially influence the making of the next netflix movie!

**Background**

In a world where data has become the new currency, businesses are leveraging big data to make informed decisions that drive success and profitability. The entertainment industry, being no exception, utilizes data analytics to comprehend market trends, audience preferences, and the performance of films based on various parameters such as director, genre, stars involved, etc. IMDb stands as a goldmine of such data, offering intricate details of almost every movie ever made.

**Task**

Your task is to create a Python script using `BeautifulSoup` and `pandas` to scrape IMDb movie data based on user ratings and release dates. This script should be able to filter movies with ratings above a certain threshold and within a specified date range.

**Expected Outcome**

- A function named `scrape_imdb` that takes four parameters: `title_type`,`user_rating`, `start_date`, and `end_date`.
- The function should return a DataFrame with the following columns:
  - **Movie Nr**: The number representing the movie’s position in the list.
  - **Title**: The title of the movie.
  - **Year**: The year the movie was released.
  - **Rating**: The IMDb rating of the movie.
  - **Runtime (min)**: The duration of the movie in minutes.
  - **Genre**: The genre of the movie.
  - **Description**: A brief description of the movie.
  - **Director**: The director of the movie.
  - **Stars**: The main stars of the movie.
  - **Votes**: The number of votes the movie received.
  - **Gross ($M)**: The gross earnings of the movie in millions of USD.

You will execute this script to scrape data for movies with the Title Type `Feature Film` that have a user rating of `7.5 and above` and were released between `January 1, 1990, and December 31, 1992`.

Remember to experiment with different title types, dates and ratings to ensure your code is versatile and can handle various searches effectively!

**Resources**

- [Beautiful Soup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Pandas Documentation](https://pandas.pydata.org/pandas-docs/stable/index.html)
- [IMDb Advanced Search](https://www.imdb.com/search/title/)


**Hint**

Your first mission is to familiarize yourself with the IMDb advanced search page. Head over to [IMDb advanced search](https://www.imdb.com/search/title/) and input the following parameters, keeping all other fields to their default values or blank:

- **Title Type**: Feature film
- **Release date**: From 1990 to 1992 (Note: You don't need to specify the day and month)
- **User Rating**: 7.5 to -

Upon searching, you'll land on a page showcasing a list of movies, each displaying vital details such as the title, release year, and crew information. Your task is to scrape this treasure trove of data.

Carefully examine the resulting URL and construct your own URL to include all the necessary parameters for filtering the movies.


---

**Best of luck! Immerse yourself in the world of movies and may the data be with you!**

**Important note**:

In the fast-changing online world, websites often get updates and make changes. When you try this lab, the IMDb website might be different from what we expect.

If you run into problems because of these changes, like new rules or things that stop you from getting data, don't worry! Instead, get creative.

You can choose another website that interests you and is good for scraping data. Websites like Wikipedia or The New York Times are good options. The main goal is still the same: get useful data and learn how to scrape it from a website that you find interesting. It's a chance to practice your web scraping skills and explore a source of information you like.

In [27]:
# Your solution goes here
!pip install requests beautifulsoup4 pandas
!pip install lxml


In [43]:
import requests
from bs4 import BeautifulSoup
import pandas as pd



In [ ]:
import requests

url = "https://www.imdb.com/search/title/?title_type=feature&user_rating=7.5,&release_date=1990,1992"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'
}
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')
print(soup.prettify())
print(response.headers['Content-Type'])


In [44]:
def scrape_imdb(title_type, user_rating, start_date, end_date):
    # Construct the URL based on the function parameters
    url = f"https://www.imdb.com/search/title/?title_type={title_type}&user_rating={user_rating},&release_date={start_date},{end_date}"

    # Send a GET request
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find all movie containers in the page
    movies = soup.find_all('div', class_='lister-item mode-advanced')

    # Create lists to store the scraped data
    titles = []
    years = []
    ratings = []
    runtimes = []
    genres = []
    descriptions = []
    directors = []
    stars = []
    votes = []
    grosses = []

    # Loop through each container to extract data
    for movie in movies:
        title = movie.h3.a.text
        year = movie.h3.find('span', class_='lister-item-year').text
        rating = movie.find('div', class_='inline-block ratings-imdb-rating').strong.text
        runtime = movie.find('span', class_='runtime').text if movie.find('span', class_='runtime') else 'N/A'
        genre = movie.find('span', class_='genre').text.strip()
        description = movie.find_all('p', class_='text-muted')[1].text.strip()
        crew = movie.find('p', '').text.split('|')
        director = crew[0].replace('Director:', '').strip() if 'Director:' in crew[0] else 'N/A'
        star = crew[1].replace('Stars:', '').strip() if len(crew) > 1 and 'Stars:' in crew[1] else 'N/A'
        vote = movie.find('span', attrs={'name': 'nv'}).text if movie.find('span', attrs={'name': 'nv'}) else 'N/A'
        gross = movie.find_all('span', attrs={'name': 'nv'})[1].text if len(movie.find_all('span', attrs={'name': 'nv'})) > 1 else 'N/A'

        # Append to lists
        titles.append(title)
        years.append(year)
        ratings.append(rating)
        runtimes.append(runtime)
        genres.append(genre)
        descriptions.append(description)
        directors.append(director)
        stars.append(star)
        votes.append(vote)
        grosses.append(gross)

    # Create a DataFrame
    movies_df = pd.DataFrame({
        'Title': titles,
        'Year': years,
        'Rating': ratings,
        'Runtime (min)': runtimes,
        'Genre': genres,
        'Description': descriptions,
        'Director': directors,
        'Stars': stars,
        'Votes': votes,
        'Gross ($M)': grosses
    })

    return movies_df

In [45]:
title_type = 'feature'
user_rating = '7.5,'
start_date = '1990-01-01'
end_date = '1992-12-31'
df = scrape_imdb(title_type, user_rating, start_date, end_date)
print(df.head())


Empty DataFrame
Columns: [Title, Year, Rating, Runtime (min), Genre, Description, Director, Stars, Votes, Gross ($M)]
Index: []


## BONUS

The search results span multiple pages, housing a total of 631 movies in our example with each page displaying 50 movies at most. To scrape data seamlessly from all pages, you'll need to dive deep into the structure of the URLs generated with each "Next" click.

Take a close look at the following URLs:
- First page:
  ```
  https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,
  ```
- Second page:
  ```
  https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,&start=51&ref_=adv_nxt
  ```
- Third page:
  ```
  https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,&start=101&ref_=adv_nxt
  ```

You should notice a pattern. There is a `start` parameter incrementing by 50 with each page, paired with a constant `ref_` parameter holding the value "adv_nxt".

Modify your script so it's capable of iterating over all available pages to fetch data on all the 631 movies (631 is the total number of movies in the proposed example).

In [ ]:
# Your solution goes here